In [1]:
import pandas as pd
import numpy as np
import re
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore")

In [1]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
tic = time.time()
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']

invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

oversample = True

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)
    
print("Used time: %d s" % (time.time()-tic))

Used time: 16 s


In [3]:
tic = time.time()
tmp = data['邀请创建时间'].apply(lambda x : x.split('-'))
data['邀请创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['邀请创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))
data['邀请创建时间_DH'] = data['邀请创建时间_D']*24 + data['邀请创建时间_H']

data['邀请是否被回答'] = data['邀请是否被回答'].fillna(0)
tmp = data[['用户ID', '邀请创建时间_DH','邀请是否被回答']].groupby(['用户ID', '邀请创建时间_DH'])['邀请是否被回答'].sum().reset_index()
tmp.columns = ['用户ID', '邀请创建时间_DH', '邀请是否被回答']
tmp = tmp.sort_values(by=['用户ID','邀请创建时间_DH'])
tmp['用户ID曾经回答数'] = tmp.groupby('用户ID')['邀请是否被回答'].cumsum() - tmp['邀请是否被回答']
tmp = tmp.sort_values(by=['用户ID','邀请创建时间_DH'], ascending=False)
    
data = data.merge(tmp, 'left', ['用户ID','邀请创建时间_DH'])
tmp = data[['用户ID曾经回答数']]
print("Used time: %d s" % (time.time()-tic))

Used time: 99 s


In [4]:
tmp[:len(invite_info)+len(invite_info_evaluate_B)].to_pickle(os.path.join(FEAT_PATH, 'user_last_answers.pickle'))